In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/MyDrive')


Mounted at /content/drive/


In [ ]:
cd CZ4034/

/content/drive/MyDrive/CZ4034


In [ ]:
import pandas as pd

pd.options.display.max_colwidth = 1000

train_df=pd.read_csv('shared_train_set.csv')
test_df=pd.read_csv('shared_test_set.csv')

print(len(train_df[train_df.label == 0]))
print(len(train_df[train_df.label == 1]))
print(len(train_df[train_df.label == 2]))

390
422
377


In [ ]:
import numpy as np
import regex as re
import nltk
from joblib import dump, load
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
from sklearn.utils import shuffle
neutral_train = train_df[train_df.label == 0]
neutral_test = test_df[test_df.label == 0]
pos_neg_train = train_df[train_df.label != 0].sample(390, random_state=1000)
pos_neg_test = test_df[test_df.label != 0].sample(110, random_state=1000)

pos_neg_train['label'] = pos_neg_train['label'].apply(neutral)
pos_neg_test['label'] = pos_neg_test['label'].apply(neutral)

train_df = pd.concat([pos_neg_train, neutral_train])
test_df = pd.concat([pos_neg_test, neutral_test])

train_df = shuffle(train_df, random_state=1000)

train_df.to_csv('shared_data_train_subj.csv')
test_df.to_csv('shared_data_test_subj.csv')

print(len(train_df[train_df.label == 0]))
print(len(train_df[train_df.label == 1]))
print(len(test_df[test_df.label == 0]))
print(len(test_df[test_df.label == 1]))

train_df[100:105]

390
390
110
110


text  label
617                                                                                                              Best company in usa.good benefits. I recomende this company for be the best at all thing and aspecs..was good sperience.....and we have layoff beacuse move to paso..texas..was realy good company in florida.      1
686                                                                                                                                                                 Itâs a delivery job. Depends on the neighborhoods that you are delivering to. They are helpful and usually flexible with hours and schedule when needed.       0
837                                                                                                                                                                           Know who you are working for. The diversity spill is bull. Management mostly non-blacks. Something to get by if you donât care about semantics.      0
580                                                                                                                                                             Educational, Positive Work Environment, Great Training, Management is fair, the most enjoy able part of the job is at will, company really cares for employees.      1
195  I currently work here itâs not bad I just hate that Iâm timed in the register and we get hours based on how good we are.You basically have to be fast in the floor and in the register mostly when the main managers are there or they will really stress you out about going faster which will make you have anxiety.      1

In [ ]:
import re

def alphanum_cleaning(x):
    x = str(x)
    pattern = '[^a-zA-Z\ ]'
    x = re.sub(pattern,'',x)
    return x.lower().strip()  

train_df['text'] = train_df['text'].apply(alphanum_cleaning)
test_df['text'] = test_df['text'].apply(alphanum_cleaning)
train_df[100:105]

text  label
617                                                                                                                    best company in usagood benefits i recomende this company for be the best at all thing and aspecswas good sperienceand we have layoff beacuse move to pasotexaswas realy good company in florida      1
686                                                                                                                                                                its a delivery job depends on the neighborhoods that you are delivering to they are helpful and usually flexible with hours and schedule when needed      0
837                                                                                                                                                                           know who you are working for the diversity spill is bull management mostly nonblacks something to get by if you dont care about semantics      0
580                                                                                                                                                           educational positive work environment great training management is fair the most enjoy able part of the job is at will company really cares for employees      1
195  i currently work here its not bad i just hate that im timed in the register and we get hours based on how good we areyou basically have to be fast in the floor and in the register mostly when the main managers are there or they will really stress you out about going faster which will make you have anxiety      1

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def stemming(x):
    stemmer = PorterStemmer()
    x = word_tokenize(x)
    result = ''
    for i in x:
        result += stemmer.stem(i) + ' '
        
    return result

train_df['stemmed_text'] = train_df['text'].apply(stemming)
test_df['stemmed_text'] = test_df['text'].apply(stemming)

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.stem import WordNetLemmatizer
def lemmatization(corpus):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]
  
train_df['lemmatized_text'] = lemmatization(train_df['text'])
test_df['lemmatized_text'] = lemmatization(test_df['text'])

In [ ]:
# Get english stopwords
stop = stopwords.words('english')
additional_stopwords = ["'s","...","'ve","``","''","'m",'--',"'ll","'d"]
stop = set(stop + additional_stopwords)
def remove_stop(x):
    x = word_tokenize(x)
    store = ''
    
    for i in x:
        if i not in stop:
            store += i + ' '
            
    return store
  
train_df['stemmed_text_clean'] = train_df['stemmed_text'].apply(remove_stop)
train_df['lemmatized_text_clean'] = train_df['lemmatized_text'].apply(remove_stop)
train_df['text_cleaned'] = train_df['text'].apply(remove_stop)

test_df['stemmed_text_clean'] = test_df['stemmed_text'].apply(remove_stop)
test_df['lemmatized_text_clean'] = test_df['lemmatized_text'].apply(remove_stop)
test_df['text_cleaned'] = test_df['text'].apply(remove_stop)

train_df[100:105]


text  ...                                                                                                                                                       text_cleaned
617                                                                                                                    best company in usagood benefits i recomende this company for be the best at all thing and aspecswas good sperienceand we have layoff beacuse move to pasotexaswas realy good company in florida  ...                best company usagood benefits recomende company best thing aspecswas good sperienceand layoff beacuse move pasotexaswas realy good company florida 
686                                                                                                                                                                its a delivery job depends on the neighborhoods that you are delivering to they are helpful and usually flexible with hours and schedule when needed  ...                                                                      delivery job depends neighborhoods delivering helpful usually flexible hours schedule needed 
837                                                                                                                                                                           know who you are working for the diversity spill is bull management mostly nonblacks something to get by if you dont care about semantics  ...                                                                   know working diversity spill bull management mostly nonblacks something get dont care semantics 
580                                                                                                                                                           educational positive work environment great training management is fair the most enjoy able part of the job is at will company really cares for employees  ...                                           educational positive work environment great training management fair enjoy able part job company really cares employees 
195  i currently work here its not bad i just hate that im timed in the register and we get hours based on how good we areyou basically have to be fast in the floor and in the register mostly when the main managers are there or they will really stress you out about going faster which will make you have anxiety  ...  currently work bad hate im timed register get hours based good areyou basically fast floor register mostly main managers really stress going faster make anxiety 

[5 rows x 7 columns]

In [ ]:
# comparison between different text preprocessing

from sklearn.model_selection import train_test_split, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from time import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

cols = ['text', 'stemmed_text', 'lemmatized_text', 'stemmed_text_clean', 'lemmatized_text_clean', 'text_cleaned']

for col_selected in cols:


  x_train, y_train = train_df[col_selected], train_df['label']
  x_test, y_test = test_df[col_selected], test_df['label']


  vectorizer = CountVectorizer(ngram_range=(1,2))
  comment_matrix = vectorizer.fit_transform(x_train)

  comment_classifier = MultinomialNB().fit(comment_matrix, y_train)
  
  start = time()
  test_vector = vectorizer.transform(x_test)
  result = comment_classifier.predict(test_vector)
  end = time()
  pred_x = comment_classifier.predict(comment_matrix) # for stack ensemble
  confuse = metrics.confusion_matrix(y_test, result) 



  # for stack ensemble
  pd.DataFrame(pred_x).to_csv('bayes_pred_train_subj.csv')
  pd.DataFrame(result).to_csv('bayes_pred_test_subj.csv')

  # calculate score
  F1_score = metrics.f1_score(y_test, result)
  average_precision = metrics.average_precision_score(y_test, result)
  precision_score= metrics.precision_score(y_test, result)
  recall_score = metrics.recall_score(y_test, result)

  print(f'=========={col_selected}==========')
  print(f'accuracy: {accuracy_score(y_test, result)}')
  print('F1 score: {0:0.3f}'.format(F1_score))
  print('Precision score: {0:0.3f}'.format(precision_score))
  print('Recall score: {0:0.3f}'.format(recall_score))
  print('Average precision-recall score: {0:0.3f}'.format(
        average_precision))
  print(f"Runtime of the program is {end - start}, number of record classified is {test_vector.shape[0]}")

==========text==========
accuracy: 0.6409090909090909
F1 score: 0.626
Precision score: 0.653
Recall score: 0.600
Average precision-recall score: 0.592
Runtime of the program is 0.026386499404907227, number of record classified is 220
==========stemmed_text==========
accuracy: 0.6681818181818182
F1 score: 0.657
Precision score: 0.680
Recall score: 0.636
Average precision-recall score: 0.614
Runtime of the program is 0.023146867752075195, number of record classified is 220
==========lemmatized_text==========
accuracy: 0.6545454545454545
F1 score: 0.635
Precision score: 0.673
Recall score: 0.600
Average precision-recall score: 0.604
Runtime of the program is 0.023407697677612305, number of record classified is 220
==========stemmed_text_clean==========
accuracy: 0.5954545454545455
F1 score: 0.604
Precision score: 0.591
Recall score: 0.618
Average precision-recall score: 0.556
Runtime of the program is 0.019289731979370117, number of record classified is 220
==========lemmatized_text_clean

In [ ]:
# comparison between different text preprocessing - svm

from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from time import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

cols = ['text', 'stemmed_text', 'lemmatized_text', 'stemmed_text_clean', 'lemmatized_text_clean', 'text_cleaned']

for col_selected in cols:


  x_train, y_train = train_df[col_selected], train_df['label']
  x_test, y_test = test_df[col_selected], test_df['label']


  vectorizer = CountVectorizer(ngram_range=(1,2))
  comment_matrix = vectorizer.fit_transform(x_train)

  SVM = svm.SVC().fit(comment_matrix, y_train)
  start = time()
  test_vector = vectorizer.transform(x_test)
  result = SVM.predict(test_vector)
  end = time()
  pred_x = SVM.predict(comment_matrix) # for stack ensemble
  confuse = metrics.confusion_matrix(y_test, result)



  # calculate score
  F1_score = metrics.f1_score(y_test, result)
  average_precision = metrics.average_precision_score(y_test, result)
  precision_score= metrics.precision_score(y_test, result)
  recall_score = metrics.recall_score(y_test, result)

  print(f'=========={col_selected}==========')
  print(f'accuracy: {accuracy_score(y_test, result)}')
  print('F1 score: {0:0.3f}'.format(F1_score))
  print('Precision score: {0:0.3f}'.format(precision_score))
  print('Recall score: {0:0.3f}'.format(recall_score))
  print('Average precision-recall score: {0:0.3f}'.format(
        average_precision))
  print(f"Runtime of the program is {end - start}, number of record classified is {test_vector.shape[0]}")

==========text==========
accuracy: 0.6590909090909091
F1 score: 0.658
Precision score: 0.661
Recall score: 0.655
Average precision-recall score: 0.605
Runtime of the program is 0.1343545913696289, number of record classified is 220
==========stemmed_text==========
accuracy: 0.65
F1 score: 0.645
Precision score: 0.654
Recall score: 0.636
Average precision-recall score: 0.598
Runtime of the program is 0.13166141510009766, number of record classified is 220
==========lemmatized_text==========
accuracy: 0.6636363636363637
F1 score: 0.667
Precision score: 0.661
Recall score: 0.673
Average precision-recall score: 0.608
Runtime of the program is 0.13066887855529785, number of record classified is 220
==========stemmed_text_clean==========
accuracy: 0.6045454545454545
F1 score: 0.595
Precision score: 0.610
Recall score: 0.582
Average precision-recall score: 0.564
Runtime of the program is 0.07391023635864258, number of record classified is 220
==========lemmatized_text_clean==========
accuracy

In [ ]:
# run propocessing as text
from sklearn.model_selection import train_test_split, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from time import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

col_selected = 'text'

x_train, y_train = train_df[col_selected], train_df['label']
x_test, y_test = test_df[col_selected], test_df['label']


vectorizer = CountVectorizer(ngram_range=(1,2))
comment_matrix = vectorizer.fit_transform(x_train)


comment_classifier = MultinomialNB().fit(comment_matrix, y_train)

start = time()
test_vector = vectorizer.transform(x_test)
result = comment_classifier.predict(test_vector)
end = time()
pred_x = comment_classifier.predict(comment_matrix) # for stack ensemble
confuse = metrics.confusion_matrix(y_test, result) 



# for stack ensemble
pd.DataFrame(pred_x).to_csv('bayes_pred_train_subj.csv')
pd.DataFrame(result).to_csv('bayes_pred_test_subj.csv')

# calculate score
F1_score = metrics.f1_score(y_test, result)
average_precision = metrics.average_precision_score(y_test, result)
precision_score= metrics.precision_score(y_test, result)
recall_score = metrics.recall_score(y_test, result)

print(f'=========={col_selected}==========')
print(f'accuracy: {accuracy_score(y_test, result)}')
print('F1 score: {0:0.3f}'.format(F1_score))
print('Precision score: {0:0.3f}'.format(precision_score))
print('Recall score: {0:0.3f}'.format(recall_score))
print('Average precision-recall score: {0:0.3f}'.format(
      average_precision))
print(f"Runtime of the program is {end - start}, number of record classified is {test_vector.shape[0]} \naverage number of record per second is {test_vector.shape[0]/(end-start)}")

==========text==========
accuracy: 0.6409090909090909
F1 score: 0.626
Precision score: 0.653
Recall score: 0.600
Average precision-recall score: 0.592
Runtime of the program is 0.02646183967590332, number of record classified is 220 
average number of record per second is 8313.858850877114


In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
nfolds=5

Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.05, 0.1, 0.15, 0.20, 0.25]
degrees = [0, 1, 2, 3, 4, 5, 6]
kernels = ['rbf', 'linear', 'poly']
param_grid = {'C': Cs, 'gamma' : gammas, 'degree' : degrees, 'kernel' : kernels}

grid_search = GridSearchCV(svm.SVC(), param_grid, cv = nfolds)
grid_search.fit(comment_matrix, y_train)
grid_search.best_params_


{'C': 0.1, 'degree': 1, 'gamma': 0.25, 'kernel': 'poly'}

In [ ]:
# using text: {'C': 0.01, 'degree': 0, 'gamma': 0.05, 'kernel': 'linear'} 0.67
# using stemmed text: {'C': 0.1, 'degree': 1, 'gamma': 0.25, 'kernel': 'poly'}, 0.65

from sklearn import svm
from sklearn.model_selection import GridSearchCV
best_params_ = {'C': 0.01, 'degree': 0, 'gamma': 0.05, 'kernel': 'linear'}
SVM = svm.SVC(**best_params_).fit(comment_matrix, y_train)

start = time()
test_vector = vectorizer.transform(x_test)
result = SVM.predict(test_vector)
end = time()
confuse = metrics.confusion_matrix(y_test, result)

# for stack ensemble
pred_x = SVM.predict(comment_matrix) # for stack ensemble
pd.DataFrame(pred_x).to_csv('svm_pred_train_subj.csv')
pd.DataFrame(result).to_csv('svm_pred_test_subj.csv')


F1_score = metrics.f1_score(y_test, result)
average_precision = metrics.average_precision_score(y_test, result)
precision_score= metrics.precision_score(y_test, result)
recall_score = metrics.recall_score(y_test, result)

print(f'=========={col_selected}==========')
print(f'accuracy: {accuracy_score(y_test, result)}')
print('F1 score: {0:0.3f}'.format(F1_score))
print('Precision score: {0:0.3f}'.format(precision_score))
print('Recall score: {0:0.3f}'.format(recall_score))
print('Average precision-recall score: {0:0.3f}'.format(
      average_precision))
print(f"Runtime of the program is {end - start}, number of record classified is {test_vector.shape[0]} \naverage number of record per second is {test_vector.shape[0]/(end-start)}")

==========text==========
accuracy: 0.6681818181818182
F1 score: 0.660
Precision score: 0.676
Recall score: 0.645
Average precision-recall score: 0.614
Runtime of the program is 0.12195920944213867, number of record classified is 220 
average number of record per second is 1803.8818143075534


In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier

# A parameter grid for XGBoost
# can add more range
params = {
    'eta': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'min_child_weight': [1, 5, 10],
}

xgb = XGBClassifier()

folds = 5

grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='accuracy', cv=folds, verbose=3)
grid.fit(comment_matrix, y_train)


Fitting 3 folds for each of 243 candidates, totalling 729 fits
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5, score=0.550, total=   1.9s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s


[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5, score=0.608, total=   1.8s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.6s remaining:    0.0s


[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5, score=0.600, total=   1.8s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7 
[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7, score=0.604, total=   1.7s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7 
[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7, score=0.631, total=   1.7s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7 
[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7, score=0.619, total=   1.7s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=1.0 
[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=1.0, score=0.642, total=   1.7s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=1.0 
[CV]  cols

[Parallel(n_jobs=1)]: Done 729 out of 729 | elapsed: 43.6min finished



 Best estimator:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1.0, verbosity=1)

 Best score:


AttributeError: ignored

In [ ]:
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best parameters:')
print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
results.to_csv('xgb-grid-search-results.csv', index=False)


 Best estimator:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1.0, verbosity=1)

 Best parameters:
{'colsample_bytree': 0.5, 'eta': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 1.0}


In [ ]:


from xgboost import XGBClassifier
xgb = XGBClassifier(colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=1.0)
xgb.fit(comment_matrix, y_train)

start = time()
test_vector = vectorizer.transform(x_test)
result = xgb.predict(test_vector)
end = time()
confuse = metrics.confusion_matrix(y_test, result)

# for stack ensemble
pred_x = xgb.predict(comment_matrix) 
pd.DataFrame(pred_x).to_csv('xgb_pred_train_subj.csv')
pd.DataFrame(result).to_csv('xgb_pred_test_subj.csv')



F1_score = metrics.f1_score(y_test, result)
average_precision = metrics.average_precision_score(y_test, result)
precision_score= metrics.precision_score(y_test, result)
recall_score = metrics.recall_score(y_test, result)

print(f'=========={col_selected}==========')
print(f'accuracy: {accuracy_score(y_test, result)}')
print('F1 score: {0:0.3f}'.format(F1_score))
print('Precision score: {0:0.3f}'.format(precision_score))
print('Recall score: {0:0.3f}'.format(recall_score))
print('Average precision-recall score: {0:0.3f}'.format(
      average_precision))
print(f"Runtime of the program is {end - start}, number of record classified is {test_vector.shape[0]} \naverage number of record per second is {test_vector.shape[0]/(end-start)}")

dump(xgb, 'subjectivity_xgb.joblib') 

==========text==========
accuracy: 0.6272727272727273
F1 score: 0.613
Precision score: 0.637
Recall score: 0.591
Average precision-recall score: 0.581
Runtime of the program is 0.03931760787963867, number of record classified is 220 
average number of record per second is 5595.45740100661
